In [2]:
# Importar todos los módulos necesarios
%run 01_constante.ipynb
%run 02_landmarks.ipynb
%run 03_optical_flow.ipynb
%run 04_keyframe.ipynb
%run 05_texture_generator.ipynb
%run 06_video_processor.ipynb
%run 07_result_analyzer.ipynb

import os
import glob
import time

def main(video_path, output_folder="output_keyframes", display=True, use_texture_maps=True):
    """
    Función principal que ejecuta todo el pipeline de detección de micro-expresiones

    Args:
        video_path: Ruta al video a procesar
        output_folder: Carpeta donde guardar resultados
        display: Si True, muestra visualización
        use_texture_maps: Si True, genera mapas de textura para mejorar el resumen

    Returns:
        Dependiendo del valor de use_texture_maps:
        - Si False: Tupla (keyframes, keyframe_indices)
        - Si True: Tupla (keyframes, keyframe_indices, texture_maps)
    """
    print(f"Procesando video: {video_path}")
    print(f"Los resultados se guardarán en: {output_folder}")

    # Inicializar componentes
    processor = VideoProcessor()

    # Procesar video (con o sin mapas de textura según el parámetro)
    if use_texture_maps:
        print("Usando mapas de textura para mejorar la detección...")
        result = processor.process_video(video_path, output_folder, display, use_texture_maps)
        keyframes, keyframe_indices, texture_maps = result
    else:
        result = processor.process_video(video_path, output_folder, display)
        keyframes, keyframe_indices = result
        texture_maps = {}

    if not keyframes:
        print("No se detectaron keyframes")
        return result

    # Mostrar estadísticas básicas
    print(f"\nEstadísticas:")
    print(f"- Total de keyframes detectados: {len(keyframes)}")
    if len(keyframe_indices) > 1:
        gaps = [keyframe_indices[i+1] - keyframe_indices[i] for i in range(len(keyframe_indices)-1)]
        print(f"- Distancia promedio entre keyframes: {sum(gaps)/len(gaps):.2f} frames")
        print(f"- Distancia mínima entre keyframes: {min(gaps)} frames")
        print(f"- Distancia máxima entre keyframes: {max(gaps)} frames")

    # Si se usaron mapas de textura, mostrar información básica
    if use_texture_maps and texture_maps:
        main_maps = [k for k in texture_maps.keys() if k in ['DXY', 'DXZ', 'DYZ', 'DC']]
        if main_maps:
            print(f"- Mapas de textura procesados: {', '.join(main_maps)}")
    
    return result

def process_expression(sujeto_number=3, expression_name="A ver", output_base="output_keyframes", display=False, use_texture_maps=True):
    """
    Procesa todos los videos de una expresión específica para un sujeto específico
    
    Args:
        sujeto_number: Número del sujeto a procesar (ej: 3 para "SUJETO 3")
        expression_name: Nombre de la expresión a procesar (ej: "A ver")
        output_base: Carpeta base donde guardar los resultados
        display: Si True, muestra visualización en tiempo real
        use_texture_maps: Si True, utiliza mapas de textura para mejorar la detección
    
    Returns:
        Dict: Estadísticas del procesamiento
    """
    print(f"==== PROCESANDO EXPRESIÓN: {expression_name} PARA SUJETO {sujeto_number} ====")
    
    # Construir la ruta directamente con los parámetros proporcionados
    sujeto_path = f"C:\\Users\\Invitado\\Documents\\facial_expression_detector\\data_procesada\\data\\SUJETO {sujeto_number}"
    expression_path = os.path.join(sujeto_path, "señas_procesadas", expression_name)
    
    # Verificar si la ruta existe
    if not os.path.exists(expression_path):
        # Intentar con "senas_procesadas" (sin tilde)
        expression_path = os.path.join(sujeto_path, "senas_procesadas", expression_name)
        
        if not os.path.exists(expression_path):
            print(f"❌ No se encontró la ruta: {expression_path}")
            return None
    
    print(f"✅ Trabajando en la ruta: {expression_path}")
    
    # Estadísticas
    stats = {
        'expression': expression_name,
        'sujeto': f"SUJETO {sujeto_number}",
        'total_muestras': 0,
        'total_videos': 0,
        'processed_videos': 0,
        'errors': 0,
        'details': []
    }
    
    # Tiempo de inicio
    start_time = time.time()
    
    # Buscar todas las carpetas de muestras
    muestra_pattern = os.path.join(expression_path, "muestra_*")
    muestra_folders = glob.glob(muestra_pattern)
    
    if not muestra_folders:
        print(f"❌ No se encontraron carpetas de muestras en {expression_path}")
        return stats
    
    print(f"Se encontraron {len(muestra_folders)} muestras.")
    stats['total_muestras'] = len(muestra_folders)
    
    # Procesar cada muestra
    for muestra_folder in sorted(muestra_folders):
        muestra_name = os.path.basename(muestra_folder)
        print(f"\n- Procesando {muestra_name}")
        
        # Buscar videos en la carpeta de muestra
        video_files = glob.glob(os.path.join(muestra_folder, "*.mp4"))
        
        # Si no se encuentran videos, intentar con nombres predecibles
        if not video_files:
            print("  No se encontraron videos directamente. Intentando patrones de nombres comunes...")
            
            # Probar diferentes patrones de nombres
            possible_names = [
                f"video_{expression_name.lower().replace(' ', '_')}_{muestra_name}.mp4",
                f"{expression_name.lower().replace(' ', '_')}_{muestra_name}_rgb.mp4",
                f"{expression_name.lower().replace(' ', '_')}_{muestra_name}.mp4",
                f"video_{muestra_name}_{expression_name.lower().replace(' ', '_')}.mp4"
            ]
            
            for name in possible_names:
                candidate = os.path.join(muestra_folder, name)
                if os.path.exists(candidate):
                    video_files = [candidate]
                    print(f"  ✅ Encontrado video con patrón: {name}")
                    break
        
        if not video_files:
            print(f"❌ No se encontraron videos en {muestra_name}")
            continue
        
        # Crear carpeta de salida personalizada para esta muestra
        output_folder_name = f"SUJETO{sujeto_number}_{expression_name.replace(' ', '_')}_{muestra_name}"
        output_folder = os.path.join(output_base, output_folder_name)
        
        # Procesar cada video encontrado
        for video_path in video_files:
            video_name = os.path.basename(video_path)
            print(f"🎬 Procesando video: {video_name}")
            
            stats['total_videos'] += 1
            
            try:
                # Crear carpeta de salida
                os.makedirs(output_folder, exist_ok=True)
                
                # Ejecutar el procesamiento
                proc_start = time.time()
                main(video_path, output_folder=output_folder, display=display, use_texture_maps=use_texture_maps)
                proc_time = time.time() - proc_start
                
                # Registrar éxito
                print(f"✅ Video procesado exitosamente en {proc_time:.2f} segundos")
                stats['processed_videos'] += 1
                stats['details'].append({
                    'muestra': muestra_name,
                    'video': video_name,
                    'path': video_path,
                    'output': output_folder,
                    'success': True,
                    'time': proc_time
                })
                
            except Exception as e:
                print(f"❌ Error procesando {video_name}: {str(e)}")
                stats['errors'] += 1
                stats['details'].append({
                    'muestra': muestra_name,
                    'video': video_name,
                    'path': video_path,
                    'output': output_folder,
                    'success': False,
                    'error': str(e)
                })
    
    # Mostrar resumen
    total_time = time.time() - start_time
    print("\n==== RESUMEN DE PROCESAMIENTO ====")
    print(f"Expresión: {expression_name}")
    print(f"Sujeto: SUJETO {sujeto_number}")
    print(f"Total de muestras: {stats['total_muestras']}")
    print(f"Total de videos: {stats['total_videos']}")
    print(f"Videos procesados con éxito: {stats['processed_videos']}")
    print(f"Errores: {stats['errors']}")
    print(f"Tiempo total: {total_time:.2f} segundos")
    
    return stats

def process_all_subjects_expression(expression_name="A ver", output_base="output_keyframes", display=False, use_texture_maps=True):
    """
    Procesa una expresión específica para todos los sujetos disponibles
    
    Args:
        expression_name: Nombre de la expresión a procesar (ej: "A ver")
        output_base: Carpeta base donde guardar los resultados
        display: Si True, muestra visualización en tiempo real
        use_texture_maps: Si True, utiliza mapas de textura para mejorar la detección
    
    Returns:
        Dict: Estadísticas del procesamiento
    """
    print(f"==== PROCESANDO EXPRESIÓN '{expression_name}' PARA TODOS LOS SUJETOS ====")
    
    # Ruta base
    base_path = r'C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data'
    
    # Buscar carpetas de sujetos
    sujeto_pattern = os.path.join(base_path, "SUJETO *")
    sujeto_folders = glob.glob(sujeto_pattern)
    
    if not sujeto_folders:
        print(f"❌ No se encontraron carpetas de sujetos en {base_path}")
        return None
    
    # Extraer números de sujetos
    sujetos = []
    for folder in sujeto_folders:
        name = os.path.basename(folder)
        if name.startswith("SUJETO "):
            try:
                num = int(name.replace("SUJETO ", ""))
                sujetos.append(num)
            except:
                continue
    
    if not sujetos:
        print("❌ No se pudieron identificar números de sujetos")
        return None
    
    print(f"Se encontraron {len(sujetos)} sujetos: {', '.join([f'SUJETO {s}' for s in sorted(sujetos)])}")
    
    # Estadísticas globales
    all_stats = {
        'expression': expression_name,
        'total_subjects': len(sujetos),
        'total_videos': 0,
        'processed_videos': 0,
        'errors': 0,
        'subjects': []
    }
    
    # Procesar cada sujeto
    for sujeto_num in sorted(sujetos):
        print(f"\n\n==== INICIANDO PROCESAMIENTO DE SUJETO {sujeto_num} ====")
        
        # Procesar este sujeto
        stats = process_expression(
            sujeto_number=sujeto_num,
            expression_name=expression_name,
            output_base=output_base,
            display=display,
            use_texture_maps=use_texture_maps
        )
        
        if stats:
            # Actualizar estadísticas globales
            all_stats['total_videos'] += stats['total_videos']
            all_stats['processed_videos'] += stats['processed_videos']
            all_stats['errors'] += stats['errors']
            all_stats['subjects'].append({
                'sujeto': f"SUJETO {sujeto_num}",
                'total_videos': stats['total_videos'],
                'processed_videos': stats['processed_videos'],
                'errors': stats['errors']
            })
    
    # Resumen final
    print("\n==== RESUMEN GLOBAL ====")
    print(f"Expresión procesada: {expression_name}")
    print(f"Total de sujetos: {all_stats['total_subjects']}")
    print(f"Total de videos: {all_stats['total_videos']}")
    print(f"Videos procesados con éxito: {all_stats['processed_videos']}")
    print(f"Errores: {all_stats['errors']}")
    
    # Detalles por sujeto
    print("\nResumen por sujeto:")
    for subject_stat in all_stats['subjects']:
        print(f"- {subject_stat['sujeto']}: {subject_stat['processed_videos']}/{subject_stat['total_videos']} videos procesados, {subject_stat['errors']} errores")
    
    return all_stats

# Ejemplo de uso
if __name__ == "__main__":
    # Carpeta base donde guardar los resultados
    output_base = r'C:\Users\Invitado\Documents\facial_expression_detector\data\outpu'
    
    # Opciones
    print("¿Qué deseas procesar?")
    print("1. Una expresión específica para un sujeto específico")
    print("2. Una expresión específica para todos los sujetos")
    
    choice = input("Elige una opción (1/2): ")
    
    # Obtener información de la expresión
    expression = input("Nombre de la expresión a procesar (ej: 'A ver'): ")
    
    if choice == "1":
        # Procesar un sujeto específico
        sujeto_num = int(input("Número del sujeto a procesar (ej: 3 para 'SUJETO 3'): "))
        process_expression(
            sujeto_number=sujeto_num,
            expression_name=expression,
            output_base=output_base,
            display=False,
            use_texture_maps=True
        )
    else:
        # Procesar todos los sujetos
        process_all_subjects_expression(
            expression_name=expression,
            output_base=output_base,
            display=False,
            use_texture_maps=True
        )

¿Qué deseas procesar?
1. Una expresión específica para un sujeto específico
2. Una expresión específica para todos los sujetos
==== PROCESANDO EXPRESIÓN: A ver PARA SUJETO 12 ====
✅ Trabajando en la ruta: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 12\senas_procesadas\A ver
Se encontraron 5 muestras.

- Procesando muestra_1
🎬 Procesando video: video_a_ver_muestra_1.mp4
Procesando video: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 12\senas_procesadas\A ver\muestra_1\video_a_ver_muestra_1.mp4
Los resultados se guardarán en: C:\Users\Invitado\Documents\facial_expression_detector\data\outpu\SUJETO12_A_ver_muestra_1
Solapamiento entre frente_cejas y ojos: 8 puntos
Solapamiento entre frente_cejas y nariz: 1 puntos
Solapamiento entre frente_cejas y mejillas: 5 puntos
Solapamiento entre frente_cejas y boca: 6 puntos
Solapamiento entre frente_cejas y mandibula: 4 puntos
Solapamiento entre frente_cejas y arrugas: 6 punto

In [ ]:
# Importar todos los módulos necesarios
%run 01_constante.ipynb
%run 02_landmarks.ipynb
%run 03_optical_flow.ipynb
%run 04_keyframe.ipynb
%run 05_texture_generator.ipynb
%run 06_video_processor.ipynb
%run 07_result_analyzer.ipynb

import os
import cv2
import time
import numpy as np
import glob
from collections import deque

def procesar_video_individual(video_path, output_folder, display=False, use_texture_maps=True):
    """
    Procesa un único video para detectar keyframes y los guarda como imágenes individuales.
    
    Args:
        video_path: Ruta al video a procesar
        output_folder: Carpeta donde guardar los keyframes
        display: Si True, muestra visualización durante el procesamiento
        use_texture_maps: Si True, genera mapas de textura durante el procesamiento
    
    Returns:
        Dict: Estadísticas del procesamiento
    """
    # Verificar que el video existe
    if not os.path.exists(video_path):
        print(f"❌ Error: El video no existe: {video_path}")
        return None

    # Crear directorio de salida si no existe
    try:
        os.makedirs(output_folder, exist_ok=True)
        print(f"✅ Carpeta de salida creada/verificada: {output_folder}")
    except Exception as e:
        print(f"❌ Error al crear directorio de salida: {e}")
        return None

    print(f"Procesando video: {video_path}")
    print(f"Los resultados se guardarán en: {output_folder}")

    # Inicializar componentes necesarios
    landmarks_detector = FacialLandmarksDetector()
    flow_tracker = OpticalFlowTracker()
    keyframe_detector = KeyFrameDetector()
    
    # Inicializar buffers para la ventana deslizante
    frames_buffer = deque(maxlen=WINDOW_SIZE)
    landmarks_buffer = deque(maxlen=WINDOW_SIZE)
    movement_values = deque(maxlen=WINDOW_SIZE)
    
    # Para almacenar keyframes detectados
    keyframes = []
    keyframe_indices = []
    frames_ya_seleccionados = set()
    
    # Abrir el video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("❌ Error al abrir el video")
        return None
    
    # Obtener información del video
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"FPS: {fps}, Total frames: {total_frames}")
    
    # Configurar optimización de velocidad
    frame_skip = max(1, int(fps / 60))  # Procesar un subconjunto de frames
    print(f"Procesando 1 de cada {frame_skip} frames para optimizar velocidad")
    
    # Variables de control
    frame_count = 0
    global_keyframe_count = 0
    summary_frames = []
    temporal_indices = []
    
    if display:
        display_interval = 5
    else:
        display_interval = 0
    
    start_time = time.time()
    
    # Bucle principal de procesamiento
    while cap.isOpened():
        # Optimización: Saltar frames para acelerar procesamiento
        if frame_skip > 1 and frame_count % frame_skip != 0:
            ret = cap.grab()  # Solo avanzar sin decodificar
            if not ret:
                break
            frame_count += 1
            continue
        
        ret, frame = cap.read()
        if not ret:
            break
        
        # Actualizar buffer de frames
        frames_buffer.append(frame.copy())
        
        # Detectar landmarks y calcular flujo óptico
        output_frame, landmarks, movement = flow_tracker.track_points(frame, landmarks_detector)
        landmarks_buffer.append(landmarks)
        movement_values.append(movement)
        
        # Detectar keyframes cuando el buffer está lleno
        if len(frames_buffer) == WINDOW_SIZE:
            # Verificar si hay un pico de movimiento que indique un keyframe
            is_peak = flow_tracker.is_key_frame(movement)
            
            if is_peak:
                # Detectar keyframes usando el detector
                keyframes_detected = keyframe_detector.select_key_frames(
                    list(frames_buffer),
                    list(landmarks_buffer),
                    list(movement_values)
                )
                
                if keyframes_detected:
                    print(f"Keyframes detectados en torno al frame {frame_count}")
                    
                    # IMPORTANTE: Este es el punto crítico para guardar los keyframes
                    for i, kf in enumerate(keyframes_detected):
                        # Calcular posición temporal
                        frame_idx_temporal = frame_count - (WINDOW_SIZE - i) + 1
                        
                        # Evitar duplicados
                        if frame_idx_temporal in frames_ya_seleccionados:
                            continue
                        
                        # Marcar como seleccionado
                        frames_ya_seleccionados.add(frame_idx_temporal)
                        
                        # GUARDAR LA IMAGEN DEL KEYFRAME - PUNTO CRÍTICO
                        keyframe_path = os.path.join(output_folder, f"keyframe_{global_keyframe_count}_tiempo_{frame_idx_temporal}.jpg")
                        print(f"💾 Guardando keyframe en: {keyframe_path}")
                        
                        # Intentar dos métodos para asegurar que se guarde
                        success = cv2.imwrite(keyframe_path, kf)
                        if not success:
                            # Método alternativo si falla el primero
                            _, buffer = cv2.imencode('.jpg', kf)
                            with open(keyframe_path, 'wb') as f:
                                f.write(buffer)
                                print(f"  Usado método alternativo para guardar")
                        
                        # Verificar que el archivo se creó
                        if os.path.exists(keyframe_path):
                            print(f"  ✅ Keyframe guardado correctamente: {os.path.getsize(keyframe_path)} bytes")
                        else:
                            print(f"  ❌ ERROR: El keyframe no se guardó correctamente")
                        
                        # Agregar a las listas
                        keyframes.append(kf)
                        keyframe_indices.append(frame_idx_temporal)
                        summary_frames.append(kf)
                        temporal_indices.append(frame_idx_temporal)
                        
                        global_keyframe_count += 1
        
        # Mostrar visualización si está activada
        if display and (frame_count % display_interval == 0):
            progress = int(100 * frame_count / total_frames)
            cv2.putText(output_frame, f"Progreso: {progress}%", 
                       (10, output_frame.shape[0] - 20), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            cv2.imshow('Análisis de Expresiones Faciales', output_frame)
            key = cv2.waitKey(1) & 0xFF
            if key == 27:  # ESC para salir
                break
        
        frame_count += 1
    
    # Liberar recursos
    cap.release()
    if display:
        cv2.destroyAllWindows()
    
    # Crear video resumen si hay keyframes
    if summary_frames and temporal_indices:
        crear_video_resumen(summary_frames, output_folder, fps, temporal_indices)
    
    # Crear estadísticas
    proc_time = time.time() - start_time
    stats = {
        'video_path': video_path,
        'output_folder': output_folder,
        'frames_procesados': frame_count,
        'keyframes_detectados': global_keyframe_count,
        'tiempo_procesamiento': proc_time,
        'fps_procesamiento': frame_count / proc_time,
        'indices_keyframes': keyframe_indices
    }
    
    # Mostrar resumen final
    print(f"\nEstadísticas:")
    print(f"- Total de keyframes detectados: {global_keyframe_count}")
    if len(keyframe_indices) > 1:
        gaps = [keyframe_indices[i+1] - keyframe_indices[i] for i in range(len(keyframe_indices)-1)]
        print(f"- Distancia promedio entre keyframes: {sum(gaps)/len(gaps):.2f} frames")
        print(f"- Distancia mínima entre keyframes: {min(gaps)} frames")
        print(f"- Distancia máxima entre keyframes: {max(gaps)} frames")
    
    print(f"✅ Video procesado exitosamente en {proc_time:.2f} segundos")
    print(f"Velocidad promedio: {frame_count/(proc_time):.2f} frames por segundo")
    print(f"Se guardaron {global_keyframe_count} keyframes en {output_folder}")
    
    return stats

def crear_video_resumen(frames, output_folder, original_fps, temporal_indices=None):
    """
    Crea un video resumen con los keyframes detectados.
    """
    if not frames:
        print("No hay frames para crear el resumen")
        return
    
    height, width = frames[0].shape[:2]
    output_path = os.path.join(output_folder, "resumen_fluido.avi")
    
    # Usar codec compatible
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    
    if temporal_indices:
        # Ordenar frames por índice temporal
        frames_con_indices = list(zip(frames, temporal_indices))
        frames_con_indices.sort(key=lambda x: x[1])
        
        # Filtrar frames demasiado cercanos
        frames_filtrados = []
        indices_filtrados = []
        ultimo_indice = -10
        
        for frame, indice in frames_con_indices:
            if indice - ultimo_indice >= 3:
                frames_filtrados.append(frame)
                indices_filtrados.append(indice)
                ultimo_indice = indice
        
        frames = frames_filtrados
        temporal_indices = indices_filtrados
    
    # Crear video con transiciones
    output_fps = original_fps / 2
    out = cv2.VideoWriter(output_path, fourcc, output_fps, (width, height))
    
    for i in range(len(frames)):
        frame_with_text = frames[i].copy()
        
        # Añadir etiqueta
        if temporal_indices and i < len(temporal_indices):
            cv2.putText(frame_with_text, f"FRAME {i+1}/{len(frames)} (Original: {temporal_indices[i]})", 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        out.write(frame_with_text)
        
        # Transiciones entre frames
        if i < len(frames) - 1:
            for t in range(1, 3):
                alpha = t / 3.0
                transicion = cv2.addWeighted(frames[i], 1-alpha, frames[i+1], alpha, 0)
                out.write(transicion)
    
    out.release()
    print(f"Video resumen creado en {output_path}")

def procesar_todas_las_muestras(ruta_base_datos, carpeta_keyframes="keyframes", display=False, use_texture_maps=True):
    """
    Procesa automáticamente todas las muestras de todas las expresiones de todos los sujetos.
    
    Args:
        ruta_base_datos: Ruta base donde están los datos
        carpeta_keyframes: Nombre de la subcarpeta donde guardar keyframes
        display: Si True, muestra visualización en tiempo real
        use_texture_maps: Si True, utiliza mapas de textura
    """
    print(f"==== PROCESANDO AUTOMÁTICAMENTE TODAS LAS MUESTRAS ====")
    print(f"Ruta base de datos: {ruta_base_datos}")
    
    # Buscar carpetas de sujetos
    sujeto_pattern = os.path.join(ruta_base_datos, "SUJETO *")
    sujeto_folders = glob.glob(sujeto_pattern)
    
    if not sujeto_folders:
        print(f"❌ No se encontraron carpetas de sujetos en {ruta_base_datos}")
        return
    
    total_videos = 0
    videos_procesados = 0
    
    # Iniciar tiempo global
    tiempo_inicio_global = time.time()
    
    # Procesar cada sujeto
    for sujeto_folder in sorted(sujeto_folders):
        sujeto_name = os.path.basename(sujeto_folder)
        print(f"\n\n{'='*50}")
        print(f"PROCESANDO SUJETO: {sujeto_name}")
        print(f"{'='*50}")
        
        # Buscar carpeta de señas procesadas
        senas_folder = os.path.join(sujeto_folder, "señas_procesadas")
        if not os.path.exists(senas_folder):
            senas_folder = os.path.join(sujeto_folder, "senas_procesadas")
            
        if not os.path.exists(senas_folder):
            print(f"❌ No se encontró carpeta de señas procesadas para {sujeto_name}")
            continue
        
        # Buscar carpetas de expresiones
        expression_folders = [f for f in glob.glob(os.path.join(senas_folder, "*")) if os.path.isdir(f)]
        
        if not expression_folders:
            print(f"❌ No se encontraron carpetas de expresiones para {sujeto_name}")
            continue
        
        # Procesar cada expresión
        for expression_folder in sorted(expression_folders):
            expression_name = os.path.basename(expression_folder)
            print(f"\n{'='*40}")
            print(f"PROCESANDO EXPRESIÓN: {expression_name}")
            print(f"{'='*40}")
            
            # Buscar carpetas de muestras
            muestra_folders = [f for f in glob.glob(os.path.join(expression_folder, "muestra_*")) if os.path.isdir(f)]
            
            if not muestra_folders:
                print(f"❌ No se encontraron carpetas de muestras para {expression_name}")
                continue
            
            # Procesar cada muestra
            for muestra_folder in sorted(muestra_folders):
                muestra_name = os.path.basename(muestra_folder)
                print(f"\n- Procesando {muestra_name}")
                
                # Buscar videos en la carpeta de muestra
                video_files = glob.glob(os.path.join(muestra_folder, "*.mp4"))
                
                # Si no se encuentran videos, intentar con nombres predecibles
                if not video_files:
                    print("  No se encontraron videos directamente. Intentando patrones de nombres comunes...")
                    
                    # Probar diferentes patrones de nombres
                    possible_names = [
                        f"video_{expression_name.lower().replace(' ', '_')}_{muestra_name}.mp4",
                        f"{expression_name.lower().replace(' ', '_')}_{muestra_name}_rgb.mp4",
                        f"{expression_name.lower().replace(' ', '_')}_{muestra_name}.mp4",
                        f"video_{muestra_name}_{expression_name.lower().replace(' ', '_')}.mp4",
                        f"video_{muestra_name}_rgb.mp4"
                    ]
                    
                    for name in possible_names:
                        candidate = os.path.join(muestra_folder, name)
                        if os.path.exists(candidate):
                            video_files = [candidate]
                            print(f"  ✅ Encontrado video con patrón: {name}")
                            break
                
                if not video_files:
                    print(f"❌ No se encontraron videos en {muestra_name}")
                    continue
                
                # Procesar cada video encontrado
                for video_path in video_files:
                    video_name = os.path.basename(video_path)
                    print(f"🎬 Procesando video: {video_name}")
                    
                    total_videos += 1
                    
                    try:
                        # Crear carpeta de keyframes dentro de la carpeta de muestra
                        output_folder = os.path.join(muestra_folder, carpeta_keyframes)
                        
                        # Procesar el video
                        video_stats = procesar_video_individual(
                            video_path=video_path,
                            output_folder=output_folder,
                            display=display,
                            use_texture_maps=use_texture_maps
                        )
                        
                        if video_stats:
                            videos_procesados += 1
                    except Exception as e:
                        print(f"❌ Error procesando {video_name}: {str(e)}")
                        import traceback
                        traceback.print_exc()
    
    # Mostrar resumen final
    tiempo_total = time.time() - tiempo_inicio_global
    print(f"\n\n{'='*50}")
    print(f"RESUMEN FINAL DEL PROCESAMIENTO AUTOMÁTICO")
    print(f"{'='*50}")
    print(f"Total de videos encontrados: {total_videos}")
    print(f"Videos procesados correctamente: {videos_procesados}")
    print(f"Tiempo total: {tiempo_total:.2f} segundos")
    
# Programa principal (punto de entrada)
if __name__ == "__main__":
    print("===== PROCESADOR AUTOMÁTICO DE KEYFRAMES DE VIDEO =====")
    print("Este script procesará automáticamente todas las muestras de todos los sujetos.\n")
    
    # Configuración:
    # 1. Ruta base de datos (modificar según sea necesario)
    ruta_base_datos = r"C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data"
    
    # 2. Nombre de la carpeta donde guardar keyframes
    carpeta_keyframes = "keyframes"
    
    # 3. Opciones de procesamiento
    display = False
    use_texture_maps = True
    
    # Mostrar configuración
    print("Configuración:")
    print(f"- Ruta base de datos: {ruta_base_datos}")
    print(f"- Carpeta de keyframes: {carpeta_keyframes}")
    print(f"- Mostrar visualización durante procesamiento: {'Sí' if display else 'No'}")
    print(f"- Usar mapas de textura: {'Sí' if use_texture_maps else 'No'}")
    
    # Iniciar procesamiento automático
    procesar_todas_las_muestras(
        ruta_base_datos=ruta_base_datos,
        carpeta_keyframes=carpeta_keyframes,
        display=display,
        use_texture_maps=use_texture_maps
    )
    
    print("\n🎉 Procesamiento automático completado.")

===== PROCESADOR AUTOMÁTICO DE KEYFRAMES DE VIDEO =====
Este script procesará automáticamente todas las muestras de todos los sujetos.

Configuración:
- Ruta base de datos: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas
- Carpeta de keyframes: keyframes_2
- Mostrar visualización durante procesamiento: No
- Usar mapas de textura: Sí
==== PROCESANDO AUTOMÁTICAMENTE TODAS LAS MUESTRAS ====
Ruta base de datos: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas
❌ No se encontraron carpetas de sujetos en C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas

🎉 Procesamiento automático completado.


In [6]:
# Importar todos los módulos necesarios
%run 01_constante.ipynb
%run 02_landmarks.ipynb
%run 03_optical_flow.ipynb
%run 04_keyframe.ipynb
%run 05_texture_generator.ipynb
%run 06_video_processor.ipynb
%run 07_result_analyzer.ipynb

import os
import cv2
import time
import numpy as np
import glob
from collections import deque

def procesar_video_individual(video_path, output_folder, display=False, use_texture_maps=True):
    """
    Procesa un único video para detectar keyframes y los guarda como imágenes individuales.
    
    Args:
        video_path: Ruta al video a procesar
        output_folder: Carpeta donde guardar los keyframes
        display: Si True, muestra visualización durante el procesamiento
        use_texture_maps: Si True, genera mapas de textura durante el procesamiento
    
    Returns:
        Dict: Estadísticas del procesamiento
    """
    # Verificar que el video existe
    if not os.path.exists(video_path):
        print(f"❌ Error: El video no existe: {video_path}")
        return None

    # Crear directorio de salida si no existe
    try:
        os.makedirs(output_folder, exist_ok=True)
        print(f"✅ Carpeta de salida creada/verificada: {output_folder}")
    except Exception as e:
        print(f"❌ Error al crear directorio de salida: {e}")
        return None

    print(f"Procesando video: {video_path}")
    print(f"Los resultados se guardarán en: {output_folder}")

    # Inicializar componentes necesarios
    landmarks_detector = FacialLandmarksDetector()
    flow_tracker = OpticalFlowTracker()
    keyframe_detector = KeyFrameDetector()
    
    # Inicializar buffers para la ventana deslizante
    frames_buffer = deque(maxlen=WINDOW_SIZE)
    landmarks_buffer = deque(maxlen=WINDOW_SIZE)
    movement_values = deque(maxlen=WINDOW_SIZE)
    
    # Para almacenar keyframes detectados
    keyframes = []
    keyframe_indices = []
    frames_ya_seleccionados = set()
    
    # Abrir el video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("❌ Error al abrir el video")
        return None
    
    # Obtener información del video
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"FPS: {fps}, Total frames: {total_frames}")
    
    # Configurar optimización de velocidad
    frame_skip = max(1, int(fps / 60))  # Procesar un subconjunto de frames
    print(f"Procesando 1 de cada {frame_skip} frames para optimizar velocidad")
    
    # Variables de control
    frame_count = 0
    global_keyframe_count = 0
    summary_frames = []
    temporal_indices = []
    
    if display:
        display_interval = 5
    else:
        display_interval = 0
    
    start_time = time.time()
    
    # Bucle principal de procesamiento
    while cap.isOpened():
        # Optimización: Saltar frames para acelerar procesamiento
        if frame_skip > 1 and frame_count % frame_skip != 0:
            ret = cap.grab()  # Solo avanzar sin decodificar
            if not ret:
                break
            frame_count += 1
            continue
        
        ret, frame = cap.read()
        if not ret:
            break
        
        # Actualizar buffer de frames
        frames_buffer.append(frame.copy())
        
        # Detectar landmarks y calcular flujo óptico
        output_frame, landmarks, movement = flow_tracker.track_points(frame, landmarks_detector)
        landmarks_buffer.append(landmarks)
        movement_values.append(movement)
        
        # Detectar keyframes cuando el buffer está lleno
        if len(frames_buffer) == WINDOW_SIZE:
            # Verificar si hay un pico de movimiento que indique un keyframe
            is_peak = flow_tracker.is_key_frame(movement)
            
            if is_peak:
                # Detectar keyframes usando el detector
                keyframes_detected = keyframe_detector.select_key_frames(
                    list(frames_buffer),
                    list(landmarks_buffer),
                    list(movement_values)
                )
                
                if keyframes_detected:
                    print(f"Keyframes detectados en torno al frame {frame_count}")
                    
                    # IMPORTANTE: Este es el punto crítico para guardar los keyframes
                    for i, kf in enumerate(keyframes_detected):
                        # Calcular posición temporal
                        frame_idx_temporal = frame_count - (WINDOW_SIZE - i) + 1
                        
                        # Evitar duplicados
                        if frame_idx_temporal in frames_ya_seleccionados:
                            continue
                        
                        # Marcar como seleccionado
                        frames_ya_seleccionados.add(frame_idx_temporal)
                        
                        # GUARDAR LA IMAGEN DEL KEYFRAME - PUNTO CRÍTICO
                        keyframe_path = os.path.join(output_folder, f"keyframe_{global_keyframe_count}_tiempo_{frame_idx_temporal}.jpg")
                        print(f"💾 Guardando keyframe en: {keyframe_path}")
                        
                        # Intentar dos métodos para asegurar que se guarde
                        success = cv2.imwrite(keyframe_path, kf)
                        if not success:
                            # Método alternativo si falla el primero
                            _, buffer = cv2.imencode('.jpg', kf)
                            with open(keyframe_path, 'wb') as f:
                                f.write(buffer)
                                print(f"  Usado método alternativo para guardar")
                        
                        # Verificar que el archivo se creó
                        if os.path.exists(keyframe_path):
                            print(f"  ✅ Keyframe guardado correctamente: {os.path.getsize(keyframe_path)} bytes")
                        else:
                            print(f"  ❌ ERROR: El keyframe no se guardó correctamente")
                        
                        # Agregar a las listas
                        keyframes.append(kf)
                        keyframe_indices.append(frame_idx_temporal)
                        summary_frames.append(kf)
                        temporal_indices.append(frame_idx_temporal)
                        
                        global_keyframe_count += 1
        
        # Mostrar visualización si está activada
        if display and (frame_count % display_interval == 0):
            progress = int(100 * frame_count / total_frames)
            cv2.putText(output_frame, f"Progreso: {progress}%", 
                       (10, output_frame.shape[0] - 20), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            cv2.imshow('Análisis de Expresiones Faciales', output_frame)
            key = cv2.waitKey(1) & 0xFF
            if key == 27:  # ESC para salir
                break
        
        frame_count += 1
    
    # Liberar recursos
    cap.release()
    if display:
        cv2.destroyAllWindows()
    
    # Crear video resumen si hay keyframes
    if summary_frames and temporal_indices:
        crear_video_resumen(summary_frames, output_folder, fps, temporal_indices)
    
    # Crear estadísticas
    proc_time = time.time() - start_time
    stats = {
        'video_path': video_path,
        'output_folder': output_folder,
        'frames_procesados': frame_count,
        'keyframes_detectados': global_keyframe_count,
        'tiempo_procesamiento': proc_time,
        'fps_procesamiento': frame_count / proc_time,
        'indices_keyframes': keyframe_indices
    }
    
    # Mostrar resumen final
    print(f"\nEstadísticas:")
    print(f"- Total de keyframes detectados: {global_keyframe_count}")
    if len(keyframe_indices) > 1:
        gaps = [keyframe_indices[i+1] - keyframe_indices[i] for i in range(len(keyframe_indices)-1)]
        print(f"- Distancia promedio entre keyframes: {sum(gaps)/len(gaps):.2f} frames")
        print(f"- Distancia mínima entre keyframes: {min(gaps)} frames")
        print(f"- Distancia máxima entre keyframes: {max(gaps)} frames")
    
    print(f"✅ Video procesado exitosamente en {proc_time:.2f} segundos")
    print(f"Velocidad promedio: {frame_count/(proc_time):.2f} frames por segundo")
    print(f"Se guardaron {global_keyframe_count} keyframes en {output_folder}")
    
    return stats

def crear_video_resumen(frames, output_folder, original_fps, temporal_indices=None):
    """
    Crea un video resumen con los keyframes detectados.
    """
    if not frames:
        print("No hay frames para crear el resumen")
        return
    
    height, width = frames[0].shape[:2]
    output_path = os.path.join(output_folder, "resumen_fluido.avi")
    
    # Usar codec compatible
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    
    if temporal_indices:
        # Ordenar frames por índice temporal
        frames_con_indices = list(zip(frames, temporal_indices))
        frames_con_indices.sort(key=lambda x: x[1])
        
        # Filtrar frames demasiado cercanos
        frames_filtrados = []
        indices_filtrados = []
        ultimo_indice = -10
        
        for frame, indice in frames_con_indices:
            if indice - ultimo_indice >= 3:
                frames_filtrados.append(frame)
                indices_filtrados.append(indice)
                ultimo_indice = indice
        
        frames = frames_filtrados
        temporal_indices = indices_filtrados
    
    # Crear video con transiciones
    output_fps = original_fps / 2
    out = cv2.VideoWriter(output_path, fourcc, output_fps, (width, height))
    
    for i in range(len(frames)):
        frame_with_text = frames[i].copy()
        
        # Añadir etiqueta
        if temporal_indices and i < len(temporal_indices):
            cv2.putText(frame_with_text, f"FRAME {i+1}/{len(frames)} (Original: {temporal_indices[i]})", 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        out.write(frame_with_text)
        
        # Transiciones entre frames
        if i < len(frames) - 1:
            for t in range(1, 3):
                alpha = t / 3.0
                transicion = cv2.addWeighted(frames[i], 1-alpha, frames[i+1], alpha, 0)
                out.write(transicion)
    
    out.release()
    print(f"Video resumen creado en {output_path}")

    """
    Procesa automáticamente todas las muestras de todas las expresiones de todos los sujetos.
    
    Args:
        ruta_base_datos: Ruta base donde están los datos
        carpeta_keyframes: Nombre de la subcarpeta donde guardar keyframes
        display: Si True, muestra visualización en tiempo real
        use_texture_maps: Si True, utiliza mapas de textura
    """
    print(f"==== PROCESANDO AUTOMÁTICAMENTE TODAS LAS MUESTRAS ====")
    print(f"Ruta base de datos: {ruta_base_datos}")
    sujeto_folder="SUJETO 2"
    total_videos = 0
    videos_procesados = 0
    
    # Iniciar tiempo global
    tiempo_inicio_global = time.time()
    
    # Buscar carpeta de señas procesadas
    senas_folder = os.path.join(sujeto_folder, "senas_procesadas")            
        
        # Buscar carpetas de expresiones
    expression_folders = [f for f in glob.glob(os.path.join(senas_folder, "*")) if os.path.isdir(f)]
        
        
        # Procesar cada expresión
    for expression_folder in sorted(expression_folders):
        expression_name = os.path.basename(expression_folder)
        print(f"\n{'='*40}")
        print(f"PROCESANDO EXPRESIÓN: {expression_name}")
        print(f"{'='*40}")
            
            # Buscar carpetas de muestras
        muestra_folders = [f for f in glob.glob(os.path.join(expression_folder, "muestra_*")) if os.path.isdir(f)]
            
        if not muestra_folders:
            print(f"❌ No se encontraron carpetas de muestras para {expression_name}")
            continue
            
            # Procesar cada muestra
        for muestra_folder in sorted(muestra_folders):
            muestra_name = os.path.basename(muestra_folder)
            print(f"\n- Procesando {muestra_name}")
                
                # Buscar videos en la carpeta de muestra
            video_files = glob.glob(os.path.join(muestra_folder, "*.mp4"))
                
                # Si no se encuentran videos, intentar con nombres predecibles
            if not video_files:
                print("  No se encontraron videos directamente. Intentando patrones de nombres comunes...")
                    
                    # Probar diferentes patrones de nombres
                possible_names = [
                    #f"video_{expression_name.lower().replace(' ', '_')}_{muestra_name}.mp4",
                    #f"{expression_name.lower().replace(' ', '_')}_{muestra_name}_rgb.mp4",
                    f"{expression_name.lower().replace(' ', '_')}_{muestra_name}.mp4",
                    #f"video_{muestra_name}_{expression_name.lower().replace(' ', '_')}.mp4",
                    #f"video_{muestra_name}_rgb.mp4"
                ]
                    
                for name in possible_names:
                    candidate = os.path.join(muestra_folder, name)
                    if os.path.exists(candidate):
                        video_files = [candidate]
                        print(f"  ✅ Encontrado video con patrón: {name}")
                        break
                
            if not video_files:
                print(f"❌ No se encontraron videos en {muestra_name}")
                continue
                
                # Procesar cada video encontrado
            for video_path in video_files:
                video_name = os.path.basename(video_path)
                print(f"🎬 Procesando video: {video_name}")
                    
                total_videos += 1
                    
                try:
                        # Crear carpeta de keyframes dentro de la carpeta de muestra
                    output_folder = os.path.join(muestra_folder, carpeta_keyframes)
                        
                        # Procesar el video
                    video_stats = procesar_video_individual(
                        video_path=video_path,
                        output_folder=output_folder,
                        display=display,
                        use_texture_maps=use_texture_maps
                    )
                        
                    if video_stats:
                         videos_procesados += 1
                except Exception as e:
                    print(f"❌ Error procesando {video_name}: {str(e)}")
                    import traceback
                    traceback.print_exc()    
        
    
    # Mostrar resumen final
    tiempo_total = time.time() - tiempo_inicio_global
    print(f"\n\n{'='*50}")
    print(f"RESUMEN FINAL DEL PROCESAMIENTO AUTOMÁTICO")
    print(f"{'='*50}")
    print(f"Total de videos encontrados: {total_videos}")
    print(f"Videos procesados correctamente: {videos_procesados}")
    print(f"Tiempo total: {tiempo_total:.2f} segundos")
def procesar_todas_las_muestras(ruta_base_datos, carpeta_keyframes="keyframes", display=False, use_texture_maps=True):
    """
    Procesa automáticamente todas las muestras usando un patrón específico para los nombres de video.
    
    Args:
        ruta_base_datos: Ruta base que ya incluye "SUJETO 2/senas_procesadas"
        carpeta_keyframes: Nombre de la subcarpeta donde guardar keyframes
        display: Si True, muestra visualización en tiempo real
        use_texture_maps: Si True, utiliza mapas de textura
    """
    print(f"==== PROCESANDO AUTOMÁTICAMENTE TODAS LAS MUESTRAS ====")
    print(f"Ruta base de datos: {ruta_base_datos}")
    
    # Verificar que la ruta base existe
    if not os.path.exists(ruta_base_datos):
        print(f"❌ La ruta base no existe: {ruta_base_datos}")
        return
    
    total_videos = 0
    videos_procesados = 0
    
    # Iniciar tiempo global
    tiempo_inicio_global = time.time()
    
    # Buscar carpetas de expresiones directamente en la ruta base
    expression_folders = [f for f in glob.glob(os.path.join(ruta_base_datos, "*")) if os.path.isdir(f)]
    
    if not expression_folders:
        print(f"❌ No se encontraron carpetas de expresiones en {ruta_base_datos}")
        return
    
    # Procesar cada expresión
    for expression_folder in sorted(expression_folders):
        expression_name = os.path.basename(expression_folder)
        print(f"\n{'='*40}")
        print(f"PROCESANDO EXPRESIÓN: {expression_name}")
        print(f"{'='*40}")
        
        # Buscar carpetas de muestras
        muestra_folders = [f for f in glob.glob(os.path.join(expression_folder, "muestra_*")) if os.path.isdir(f)]
        
        if not muestra_folders:
            print(f"❌ No se encontraron carpetas de muestras para {expression_name}")
            continue
        
        # Procesar cada muestra
        for muestra_folder in sorted(muestra_folders):
            muestra_name = os.path.basename(muestra_folder)
            print(f"\n- Procesando {muestra_name}")
            
            # Utilizar específicamente el patrón solicitado
            video_pattern = f"{expression_name.lower().replace(' ', '_')}_{muestra_name}.mp4"
            video_path = os.path.join(muestra_folder, video_pattern)
            
            # Verificar si el video existe con ese nombre exacto
            if os.path.exists(video_path):
                print(f"✅ Encontrado video: {video_pattern}")
                video_files = [video_path]
            else:
                # Si no existe con ese nombre exacto, usar un patrón más flexible
                # que permita cualquier número al final
                flexible_pattern = f"{expression_name.lower().replace(' ', '_')}_{muestra_name}*.mp4"
                video_files = glob.glob(os.path.join(muestra_folder, flexible_pattern))
                
                if video_files:
                    print(f"✅ Encontrados {len(video_files)} videos con patrón similar: {flexible_pattern}")
                else:
                    print(f"❌ No se encontró el video con patrón: {video_pattern}")
                    continue
            
            # Procesar cada video encontrado
            for video_path in video_files:
                video_name = os.path.basename(video_path)
                print(f"🎬 Procesando video: {video_name}")
                
                total_videos += 1
                
                try:
                    # Crear carpeta de keyframes dentro de la carpeta de muestra
                    output_folder = os.path.join(muestra_folder, carpeta_keyframes)
                    
                    # Procesar el video (aquí llamarías a la función que procesa el video)
                    video_stats = procesar_video_individual(
                        video_path=video_path,
                        output_folder=output_folder,
                        display=display,
                        use_texture_maps=use_texture_maps
                    )
                    
                    if video_stats:
                        videos_procesados += 1
                except Exception as e:
                    print(f"❌ Error procesando {video_name}: {str(e)}")
                    import traceback
                    traceback.print_exc()
    
    # Mostrar resumen final
    tiempo_total = time.time() - tiempo_inicio_global
    print(f"\n\n{'='*50}")
    print(f"RESUMEN FINAL DEL PROCESAMIENTO AUTOMÁTICO")
    print(f"{'='*50}")
    print(f"Total de videos encontrados: {total_videos}")
    print(f"Videos procesados correctamente: {videos_procesados}")
    print(f"Tiempo total: {tiempo_total:.2f} segundos")
    

# Programa principal (punto de entrada)
if __name__ == "__main__":
    print("===== PROCESADOR AUTOMÁTICO DE KEYFRAMES DE VIDEO =====")
    print("Este script procesará automáticamente todas las muestras de todos los sujetos.\n")
    
    # Configuración:
    # 1. Ruta base de datos (modificar según sea necesario)
    ruta_base_datos = r"C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas"
    
    # 2. Nombre de la carpeta donde guardar keyframes
    carpeta_keyframes = "keyframes_2"
    
    # 3. Opciones de procesamiento
    display = False
    use_texture_maps = True
    
    # Mostrar configuración
    print("Configuración:")
    print(f"- Ruta base de datos: {ruta_base_datos}")
    print(f"- Carpeta de keyframes: {carpeta_keyframes}")
    print(f"- Mostrar visualización durante procesamiento: {'Sí' if display else 'No'}")
    print(f"- Usar mapas de textura: {'Sí' if use_texture_maps else 'No'}")
    
    # Iniciar procesamiento automático
    procesar_todas_las_muestras(
        ruta_base_datos=ruta_base_datos,
        carpeta_keyframes=carpeta_keyframes,
        display=display,
        use_texture_maps=use_texture_maps
    )
    
    print("\n🎉 Procesamiento automático completado.")

===== PROCESADOR AUTOMÁTICO DE KEYFRAMES DE VIDEO =====
Este script procesará automáticamente todas las muestras de todos los sujetos.

Configuración:
- Ruta base de datos: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas
- Carpeta de keyframes: keyframes_2
- Mostrar visualización durante procesamiento: No
- Usar mapas de textura: Sí
==== PROCESANDO AUTOMÁTICAMENTE TODAS LAS MUESTRAS ====
Ruta base de datos: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas

PROCESANDO EXPRESIÓN: A ver

- Procesando muestra_1
✅ Encontrado video: a_ver_muestra_1.mp4
🎬 Procesando video: a_ver_muestra_1.mp4
✅ Carpeta de salida creada/verificada: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas\A ver\muestra_1\keyframes_2
Procesando video: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas\A ver\muestra_1